In [11]:
import os

In [12]:
%pwd
#to check current working directory

'D:\\ML\\MLOps Bootcamp course\\mlops\\datascienceproject'

In [13]:
os.chdir('D:\ML\MLOps Bootcamp course\mlops\datascienceproject')
#to go up one directory
#because we need to work in project directory, to execute program
%pwd

'D:\\ML\\MLOps Bootcamp course\\mlops\\datascienceproject'

In [14]:
from dataclasses import dataclass
from pathlib import Path

#by read and assign this class we'l be able to read the config.yaml file
@dataclass
class DataIngestionConfig:
    #similar as per defined in config.yaml
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    
#difference between normal class and dataclass
# 1. Boilerplate code: Dataclasses automatically generate special methods like __init__, __repr__, and __eq__.
# 2. Immutability: You can make fields immutable by setting `frozen=True`.
# 3. Default values: Dataclasses support default values for fields.
#normal class we use self keyword to access attributes
#dataclass: we focus on assigning values to fields/variables, dont have functions

#above config should pass to data_ingestion_config

In [15]:
#import file path 
#use of * means import all
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml, create_directories

In [ ]:
#misplaced code?
#import os

# Get the absolute path to the project root directory
#PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Update the constants with absolute paths
#CONFIG_FILE_PATH = os.path.join(PROJECT_ROOT, "config", "config.yaml")
#PARAMS_FILE_PATH = os.path.join(PROJECT_ROOT, "params.yaml")
#SCHEMA_FILE_PATH = os.path.join(PROJECT_ROOT, "schema.yaml")

# Convert to Path objects
#from pathlib import Path
#CONFIG_FILE_PATH = Path(CONFIG_FILE_PATH)
#PARAMS_FILE_PATH = Path(PARAMS_FILE_PATH)
#SCHEMA_FILE_PATH = Path(SCHEMA_FILE_PATH)

In [17]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root]) #it is present in config.yaml

#below function is to get details of data_ingestion from config.yaml
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir]) #because root_dir first mentioned in config.yaml
        
        data_ingestion_config = DataIngestionConfig(
            root_dir        = config.root_dir,
            source_URL      = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir       = config.unzip_dir
        )

        return data_ingestion_config

In [18]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile #for extracting zip files

In [19]:
#components data ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    #once i get the config, i can use it to download the file
    #downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                #when we use retrieval we download file in local path
                url      = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f" {filename} download! with following info: \n {headers}")
        else:
            logger.info(f" File already exists.")
    
    #extracting zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the downloaded zip file into data directory.
        function returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    #if we select config and press F12 then it will redirect from where we defined the ConfigurationManager class
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
#since we are using try and except block, we can catch the exception and log it
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\ML\\MLOps Bootcamp course\\mlops\\config\\config.yaml'